# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import countDistinct

import pyspark.sql.functions as f

from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re

import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Magic
%matplotlib inline

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()


# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
path = 'mini_sparkify_event_data.json'

user_log = spark.read.json(path)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [4]:
#Look at the schema
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
#Look at some rows using pandas for more readable output
user_log.toPandas().head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
5,The All-American Rejects,Logged In,Micah,M,81,Long,208.29995,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Don't Leave Me,200,1538352678000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
6,The Velvet Underground / Nico,Logged In,Micah,M,82,Long,260.46649,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Run Run Run,200,1538352886000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
7,Starflyer 59,Logged In,Colin,M,53,Freeman,185.44281,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Passengers (Old Album Version),200,1538352899000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
8,None,Logged In,Colin,M,54,Freeman,NaN,paid,"Bakersfield, CA",PUT,Add to Playlist,1.538173e+12,29,None,200,1538352905000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
9,Frumpies,Logged In,Colin,M,55,Freeman,134.47791,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Fuck Kitty,200,1538353084000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [6]:
#How many rows do we have?
user_log.count()

286500

In [7]:
#Find out number of unique values per column and their datatypes
columns = user_log.schema.fieldNames()
datatypes = pd.DataFrame(user_log.dtypes, columns = ['column', 'dtype'])

for col in columns:
    nunique = user_log.select(col).dropDuplicates().count()
    dtype = datatypes[datatypes['column'] == col]['dtype'].values[0]
    output = 'Column: {} ({}) has {} unique values'.format(col, dtype, nunique)
    print(output)

Column: artist (string) has 17656 unique values
Column: auth (string) has 4 unique values
Column: firstName (string) has 190 unique values
Column: gender (string) has 3 unique values
Column: itemInSession (bigint) has 1322 unique values
Column: lastName (string) has 174 unique values
Column: length (double) has 14866 unique values
Column: level (string) has 2 unique values
Column: location (string) has 115 unique values
Column: method (string) has 2 unique values
Column: page (string) has 22 unique values
Column: registration (bigint) has 226 unique values
Column: sessionId (bigint) has 2354 unique values
Column: song (string) has 58481 unique values
Column: status (bigint) has 3 unique values
Column: ts (bigint) has 277447 unique values
Column: userAgent (string) has 57 unique values
Column: userId (string) has 226 unique values


In [8]:
#Look at the top artists
user_log.groupBy("artist").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head(20)

,artist,distinct_name
0,None,58392
1,Kings Of Leon,1841
2,Coldplay,1813
3,Florence + The Machine,1236
4,Dwight Yoakam,1135
5,BjÃÂ¶rk,1133
6,The Black Keys,1125
7,Muse,1090
8,Justin Bieber,1044
9,Jack Johnson,1007


In [9]:
#Look at the top auth
user_log.groupBy("auth").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head(20)

,auth,distinct_name
0,Logged In,278102
1,Logged Out,8249
2,Guest,97
3,Cancelled,52


In [10]:
#Look at the top gender
user_log.groupBy("gender").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head(20)

,gender,distinct_name
0,F,154578
1,M,123576
2,None,8346


In [11]:
##Look at the top  itemInSession
user_log.groupBy("itemInSession").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,itemInSession,distinct_name
0,0,3278
1,1,3125
2,2,3067
3,3,3013
4,4,2977
5,5,2925
6,6,2886
7,7,2857
8,8,2816
9,9,2771


In [12]:
##Look at the top  level
user_log.groupBy("level").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,level,distinct_name
0,paid,228162
1,free,58338


In [13]:
##Look at the top  location
user_log.groupBy("location").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,location,distinct_name
0,"Los Angeles-Long Beach-Anaheim, CA",30131
1,"New York-Newark-Jersey City, NY-NJ-PA",23684
2,"Boston-Cambridge-Newton, MA-NH",13873
3,"Houston-The Woodlands-Sugar Land, TX",9499
4,None,8346
5,"Charlotte-Concord-Gastonia, NC-SC",7780
6,"Dallas-Fort Worth-Arlington, TX",7605
7,"Louisville/Jefferson County, KY-IN",6880
8,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",5890
9,"Chicago-Naperville-Elgin, IL-IN-WI",5114


In [14]:
##Look at the top method
user_log.groupBy("method").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,method,distinct_name
0,PUT,261064
1,GET,25436


In [15]:
##Look at the top pages
user_log.groupBy("page").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,page,distinct_name
0,NextSong,228108
1,Home,14457
2,Thumbs Up,12551
3,Add to Playlist,6526
4,Add Friend,4277
5,Roll Advert,3933
6,Login,3241
7,Logout,3226
8,Thumbs Down,2546
9,Downgrade,2055


In [16]:
##Look at the top registration
user_log.groupBy("registration").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head()

,registration,distinct_name
0,1.529028e+12,9632
1,NaN,8346
2,1.536404e+12,7230
3,1.536642e+12,6880
4,1.538337e+12,5732


In [17]:
##Look at the top status
user_log.groupBy("status").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,status,distinct_name
0,200,259812
1,307,26430
2,404,258


In [18]:
##Look at the top userAgent
user_log.groupBy("userAgent").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,userAgent,distinct_name
0,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",22751
1,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",19611
2,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",18448
3,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",17348
4,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,16700
5,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",15395
6,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",14598
7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,10300
8,"""Mozilla/5.0 (iPad; CPU OS 7_1_2 like Mac OS X...",8912
9,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,8624


In [19]:
##Look at the top userId
user_log.groupBy("userId").count().sort(desc("count")).toPandas().head(10)

,userId,count
0,39,9632
1,,8346
2,92,7230
3,140,6880
4,300011,5732
5,124,4825
6,300021,4659
7,300017,4428
8,85,4370
9,42,4257


In [20]:
#We appear to have several thousand rows with missinng userId let's look at those pages
missing_userIds = user_log.select(["page", "level"]).where(user_log.userId == "")
missing_userIds.groupby("page").count().sort(desc("count")).toPandas().head(10)

,page,count
0,Home,4375
1,Login,3241
2,About,429
3,Help,272
4,Register,18
5,Error,6
6,Submit Registration,5


The page names of the rows with missing Ids suggest this is probably activity prior to the user logging in.  We don't have any other kind of secondary ID, so we cannot attempt to associate these rows with an actual user.  The userAgent field isn't specific enough.  Because of this we must simply drop these rows.

In [21]:
#create user_log_valid - removing rows without userId
user_log_valid = user_log.filter(user_log["userId"] != "")

In [22]:
#We are interested in identifying when users cancel their account.  Create a function to flag this
flag_cancellation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [23]:
#Use function to flag cancellation rows
user_log_valid = user_log_valid.withColumn("cancelled", flag_cancellation_event("page"))

In [24]:
user_log_valid.toPandas().head(20)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,cancelled
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,0
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9,0
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,0
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9,0
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,0
5,The All-American Rejects,Logged In,Micah,M,81,Long,208.29995,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Don't Leave Me,200,1538352678000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9,0
6,The Velvet Underground / Nico,Logged In,Micah,M,82,Long,260.46649,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Run Run Run,200,1538352886000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9,0
7,Starflyer 59,Logged In,Colin,M,53,Freeman,185.44281,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Passengers (Old Album Version),200,1538352899000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,0
8,None,Logged In,Colin,M,54,Freeman,NaN,paid,"Bakersfield, CA",PUT,Add to Playlist,1538173362000,29,None,200,1538352905000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,0
9,Frumpies,Logged In,Colin,M,55,Freeman,134.47791,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Fuck Kitty,200,1538353084000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,0


In [25]:
#Create grouped by dataframe
users_grouped = user_log_valid.groupby("userId").sum("cancelled").sort(desc("sum(cancelled)"))#.toPandas().head(10)
#Check we have removed the missing userId
#users_grouped.count().sort(desc("count")).toPandas().head(10)
users_grouped.show()

+------+--------------+
|userId|sum(cancelled)|
+------+--------------+
|100007|             1|
|300007|             1|
|    17|             1|
|    28|             1|
|100025|             1|
|    18|             1|
|100005|             1|
|    73|             1|
|     3|             1|
|100022|             1|
|100014|             1|
|   125|             1|
|100006|             1|
|   101|             1|
|    70|             1|
|    54|             1|
|    87|             1|
|100021|             1|
|    29|             1|
|    51|             1|
+------+--------------+
only showing top 20 rows



In [26]:
users_grouped = user_log_valid.groupby("userId")\
                    .agg({"cancelled" : "sum",
                         "page" : "count",
                         "gender" : 'approx_count_distinct'})\
                    .withColumnRenamed("sum(cancelled)", "money")
users_grouped.show()

+------+-----------------------------+-----+-----------+
|userId|approx_count_distinct(gender)|money|count(page)|
+------+-----------------------------+-----+-----------+
|100010|                            1|    0|        381|
|200002|                            1|    0|        474|
|   125|                            1|    1|         11|
|    51|                            1|    1|       2464|
|   124|                            1|    0|       4825|
|     7|                            1|    0|        201|
|    54|                            1|    1|       3437|
|    15|                            1|    0|       2278|
|   155|                            1|    0|       1002|
|   132|                            1|    0|       2304|
|   154|                            1|    0|        118|
|100014|                            1|    1|        310|
|   101|                            1|    1|       2149|
|    11|                            1|    0|        848|
|   138|                       

In [27]:
#Create list of all the distinct page names
pages_list = list(user_log_valid.select("page").dropDuplicates().toPandas()["page"])
pages_list

['Cancel',
 'Submit Downgrade',
 'Thumbs Down',
 'Home',
 'Downgrade',
 'Roll Advert',
 'Logout',
 'Save Settings',
 'Cancellation Confirmation',
 'About',
 'Settings',
 'Add to Playlist',
 'Add Friend',
 'NextSong',
 'Thumbs Up',
 'Help',
 'Upgrade',
 'Error',
 'Submit Upgrade']

In [28]:
#Prepend "_event" to each page name for use as columns
pages_columns_renamed = ["event_" + x.replace(" ", "_") for x in pages_list]
pages_columns_renamed

['event_Cancel',
 'event_Submit_Downgrade',
 'event_Thumbs_Down',
 'event_Home',
 'event_Downgrade',
 'event_Roll_Advert',
 'event_Logout',
 'event_Save_Settings',
 'event_Cancellation_Confirmation',
 'event_About',
 'event_Settings',
 'event_Add_to_Playlist',
 'event_Add_Friend',
 'event_NextSong',
 'event_Thumbs_Up',
 'event_Help',
 'event_Upgrade',
 'event_Error',
 'event_Submit_Upgrade']

In [29]:
#Create dataframe of userId pivoted by page, to create a column of page views for each page 
user_page_events = user_log_valid.groupby("userId").pivot("page", pages_list).count()
#Replace NaN values with 0
user_page_events = user_page_events.fillna(0)
#Rename the columns, so each is prepended with "_event"
user_page_events = user_page_events.toDF(* ["userId"] + pages_columns_renamed)

In [30]:
user_page_events.toPandas().head(10)

,userId,event_Cancel,event_Submit_Downgrade,event_Thumbs_Down,event_Home,event_Downgrade,event_Roll_Advert,event_Logout,event_Save_Settings,event_Cancellation_Confirmation,event_About,event_Settings,event_Add_to_Playlist,event_Add_Friend,event_NextSong,event_Thumbs_Up,event_Help,event_Upgrade,event_Error,event_Submit_Upgrade
0,200002,0,0,6,20,5,7,5,0,0,3,3,8,4,387,21,2,2,0,1
1,100010,0,0,5,11,0,52,5,0,0,1,0,7,4,275,17,2,2,0,0
2,125,1,0,0,0,0,1,0,0,1,0,0,0,0,8,0,0,0,0,0
3,51,1,0,21,75,23,0,24,1,1,2,12,52,28,2111,100,12,0,1,0
4,124,0,0,41,176,41,4,59,6,0,7,20,118,74,4079,171,23,0,6,0
5,7,0,0,1,10,0,16,3,1,0,0,3,5,1,150,7,1,2,1,0
6,54,1,1,29,129,39,47,36,3,1,5,17,72,33,2841,163,17,1,1,1
7,15,0,0,14,93,28,1,27,2,0,2,16,59,31,1914,81,8,0,2,0
8,155,0,0,3,30,12,8,14,0,0,2,5,24,11,820,58,9,2,3,1
9,132,0,0,17,83,19,2,34,4,0,4,17,38,41,1928,96,16,1,3,1


In [31]:
user_page_events.toPandas().groupby("event_Cancellation_Confirmation").mean()

,event_Cancel,event_Submit_Downgrade,event_Thumbs_Down,event_Home,event_Downgrade,event_Roll_Advert,event_Logout,event_Save_Settings,event_About,event_Settings,event_Add_to_Playlist,event_Add_Friend,event_NextSong,event_Thumbs_Up,event_Help,event_Upgrade,event_Error,event_Submit_Upgrade
event_Cancellation_Confirmation,,,,,,,,,,,,,,,,,,
0,0.0,0.312139,11.849711,48.612717,9.930636,17.144509,15.450867,1.456647,2.537572,7.190751,31.722543,21.046243,1108.173410,61.803468,7.023121,2.236994,1.271676,0.734104
1,1.0,0.173077,9.538462,32.153846,6.480769,18.596154,10.634615,1.115385,1.076923,5.192308,19.961538,12.230769,699.884615,35.750000,4.596154,2.153846,0.615385,0.615385


In [32]:
user_log_valid.sort(['userId','sessionId']).toPandas().head(100)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,cancelled
0,Sea Wolf,Logged In,Brycen,M,0,Bradley,265.53424,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Ses Monuments,200,1538965220000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
1,George Baker Selection,Logged In,Brycen,M,1,Bradley,200.43710,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Morning Sky,200,1538965485000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
2,Klaus Lage,Logged In,Brycen,M,2,Bradley,208.03873,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Du Hast'n Freund In Mir (You've Got A Friend I...,200,1538965685000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
3,Dave Grusin,Logged In,Brycen,M,3,Bradley,526.44526,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Playera,200,1538965893000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
4,Blue Oyster Cult,Logged In,Brycen,M,4,Bradley,272.06485,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Burnin' For You,200,1538966419000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
5,Lifehouse,Logged In,Brycen,M,5,Bradley,207.01995,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Sky Is Falling,200,1538966691000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
6,Camper Van Beethoven,Logged In,Brycen,M,6,Bradley,106.31791,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Skinhead Stomp,200,1538966898000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
7,None,Logged In,Brycen,M,7,Bradley,NaN,paid,"Laurel, MS",PUT,Thumbs Up,1538159495000,9,None,307,1538966899000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
8,Dwight Yoakam,Logged In,Brycen,M,8,Bradley,239.30730,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,You're The One,200,1538967004000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0
9,Matthew Good Band,Logged In,Brycen,M,9,Bradley,264.85506,paid,"Laurel, MS",PUT,NextSong,1538159495000,9,Strange Days,200,1538967243000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,0


In [33]:
user_log.select(["userId", "firstname", "page", "song"]).where(user_log.userId == "1046").collect()

[]

In [34]:
user_log.groupby(["page"]).count().collect()

[Row(page='Cancel', count=52),
 Row(page='Submit Downgrade', count=63),
 Row(page='Thumbs Down', count=2546),
 Row(page='Home', count=14457),
 Row(page='Downgrade', count=2055),
 Row(page='Roll Advert', count=3933),
 Row(page='Logout', count=3226),
 Row(page='Save Settings', count=310),
 Row(page='Cancellation Confirmation', count=52),
 Row(page='About', count=924),
 Row(page='Submit Registration', count=5),
 Row(page='Settings', count=1514),
 Row(page='Login', count=3241),
 Row(page='Register', count=18),
 Row(page='Add to Playlist', count=6526),
 Row(page='Add Friend', count=4277),
 Row(page='NextSong', count=228108),
 Row(page='Thumbs Up', count=12551),
 Row(page='Help', count=1726),
 Row(page='Upgrade', count=499),
 Row(page='Error', count=258),
 Row(page='Submit Upgrade', count=159)]

In [35]:
#Pages
user_log.groupBy("page").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

,page,distinct_name
0,NextSong,228108
1,Home,14457
2,Thumbs Up,12551
3,Add to Playlist,6526
4,Add Friend,4277
5,Roll Advert,3933
6,Login,3241
7,Logout,3226
8,Thumbs Down,2546
9,Downgrade,2055


In [36]:
'''
'''
def check_columns_map_one_to_one(df, col_to_group_by, col_to_check_distincts):
    count_of_group_by_with_more_than_one = df.groupby([col_to_group_by]).\
        agg(countDistinct(col_to_check_distincts).\
        alias('unique_count')).\
        filter("unique_count > 1").count()
    return (count_of_group_by_with_more_than_one == 0)

In [37]:
#There are several columns which sound like they might map 1-1 with a userId. 
#Check each column, if it does map 1-1 it is a candidate for being used as a feature
cols_to_check_are_unique_per_userId = ['gender', 'level', 'userAgent', 'status', 'registration', 'method', 'auth', 'location']

for col in cols_to_check_are_unique_per_userId:
    unique_check = check_columns_map_one_to_one(user_log_valid, 'userId', col)
    print('Every userId has a unique value for {} : {}'.format(col, unique_check))
    
#gender, userAgent, registraion and location all map 1-1 with a userId so are all candidate features

Every userId has a unique value for gender : True
Every userId has a unique value for level : False
Every userId has a unique value for userAgent : True
Every userId has a unique value for status : False
Every userId has a unique value for registration : True
Every userId has a unique value for method : False
Every userId has a unique value for auth : False
Every userId has a unique value for location : True


In [38]:
gender_lookup = user_log_valid.groupBy("userId", "gender").count().select("userId", "gender")

is_female = udf(lambda x: 1 if x == "F" else 0, IntegerType())

gender_lookup = gender_lookup.withColumn("is_female", is_female("gender")).select("userId", "is_female")
gender_lookup.toPandas().head(200)

,userId,is_female
0,44,1
1,46,1
2,41,1
3,72,1
4,300023,1
5,39,1
6,100010,1
7,40,1
8,94,1
9,35,1


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [39]:
'''
INPUT:
df

OUPUT:
df
'''
def engineer_features(df):
    
    #Create list of all the distinct page names
    pages_list = list(user_log_valid.select("page").dropDuplicates().toPandas()["page"])
    #Prepend "_event" to each page name for use as columns
    pages_columns_renamed = ["event_" + x.replace(" ", "_") for x in pages_list]
    
    #Create dataframe of userId pivoted by page, to create a column of page views for each page 
    user_page_events = user_log_valid.groupby("userId").pivot("page", pages_list).count()
    #Replace NaN values with 0
    user_page_events = user_page_events.fillna(0)
    #Rename the columns, so each is prepended with "_event"
    user_page_events = user_page_events.toDF(* ["userId"] + pages_columns_renamed)
    
    #The cancellation confirmation event is what we are trying to predict, we rename this "label"
    user_page_events= user_page_events.withColumnRenamed("event_Cancellation_Confirmation","label")
    
    gender_lookup = df.groupBy("userId", "gender").count().select("userId", "gender")
    
   
    #create UDF to map gender (F|M) to 1|0
    is_female = udf(lambda x: 1 if x == "F" else 0, IntegerType())
    #Map gender as 1|o in is_female column, drop gender column
    gender_lookup = gender_lookup.withColumn("is_female", is_female("gender")).select("userId", "is_female")
    
    #Join our userId feature tables together
    df_features = gender_lookup.join(user_page_events, on="userId", how="inner")
    
    return df_features

In [40]:
df_features = engineer_features(user_log_valid)

In [41]:
df_features.toPandas().head(10)

,userId,is_female,event_Cancel,event_Submit_Downgrade,event_Thumbs_Down,event_Home,event_Downgrade,event_Roll_Advert,event_Logout,event_Save_Settings,...,event_About,event_Settings,event_Add_to_Playlist,event_Add_Friend,event_NextSong,event_Thumbs_Up,event_Help,event_Upgrade,event_Error,event_Submit_Upgrade
0,100010,1,0,0,5,11,0,52,5,0,...,1,0,7,4,275,17,2,2,0,0
1,200002,0,0,0,6,20,5,7,5,0,...,3,3,8,4,387,21,2,2,0,1
2,125,0,1,0,0,0,0,1,0,0,...,0,0,0,0,8,0,0,0,0,0
3,124,1,0,0,41,176,41,4,59,6,...,7,20,118,74,4079,171,23,0,6,0
4,51,0,1,0,21,75,23,0,24,1,...,2,12,52,28,2111,100,12,0,1,0
5,7,0,0,0,1,10,0,16,3,1,...,0,3,5,1,150,7,1,2,1,0
6,15,0,0,0,14,93,28,1,27,2,...,2,16,59,31,1914,81,8,0,2,0
7,54,1,1,1,29,129,39,47,36,3,...,5,17,72,33,2841,163,17,1,1,1
8,155,1,0,0,3,30,12,8,14,0,...,2,5,24,11,820,58,9,2,3,1
9,100014,0,1,0,3,7,3,2,3,0,...,0,1,7,6,257,17,2,0,0,0


In [42]:
df_features.columns

['userId',
 'is_female',
 'event_Cancel',
 'event_Submit_Downgrade',
 'event_Thumbs_Down',
 'event_Home',
 'event_Downgrade',
 'event_Roll_Advert',
 'event_Logout',
 'event_Save_Settings',
 'label',
 'event_About',
 'event_Settings',
 'event_Add_to_Playlist',
 'event_Add_Friend',
 'event_NextSong',
 'event_Thumbs_Up',
 'event_Help',
 'event_Upgrade',
 'event_Error',
 'event_Submit_Upgrade']

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [43]:
numeric_feature_columns = df_features.columns
numeric_feature_columns.remove('userId')
numeric_feature_columns.remove('label')
numeric_feature_columns

['is_female',
 'event_Cancel',
 'event_Submit_Downgrade',
 'event_Thumbs_Down',
 'event_Home',
 'event_Downgrade',
 'event_Roll_Advert',
 'event_Logout',
 'event_Save_Settings',
 'event_About',
 'event_Settings',
 'event_Add_to_Playlist',
 'event_Add_Friend',
 'event_NextSong',
 'event_Thumbs_Up',
 'event_Help',
 'event_Upgrade',
 'event_Error',
 'event_Submit_Upgrade']

In [44]:
#Assemble the Vector of Numeric features
assembler = VectorAssembler(inputCols=numeric_feature_columns, outputCol="NumFeatures")
df = assembler.transform(df_features)
#df.head() 

#Normalize the Vectors
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures")
df =  scaler.transform(df)
df.head() 

Row(userId='100010', is_female=1, event_Cancel=0, event_Submit_Downgrade=0, event_Thumbs_Down=5, event_Home=11, event_Downgrade=0, event_Roll_Advert=52, event_Logout=5, event_Save_Settings=0, label=0, event_About=1, event_Settings=0, event_Add_to_Playlist=7, event_Add_Friend=4, event_NextSong=275, event_Thumbs_Up=17, event_Help=2, event_Upgrade=2, event_Error=0, event_Submit_Upgrade=0, NumFeatures=DenseVector([1.0, 0.0, 0.0, 5.0, 11.0, 0.0, 52.0, 5.0, 0.0, 1.0, 0.0, 7.0, 4.0, 275.0, 17.0, 2.0, 2.0, 0.0, 0.0]), ScaledNumFeatures=DenseVector([0.0036, 0.0, 0.0, 0.0178, 0.0392, 0.0, 0.1852, 0.0178, 0.0, 0.0036, 0.0, 0.0249, 0.0142, 0.9792, 0.0605, 0.0071, 0.0071, 0.0, 0.0]))

In [45]:
data = df.select("label","ScaledNumFeatures").withColumnRenamed('ScaledNumFeatures','features')
#data.toPandas().head()
data.head()

Row(label=0, features=DenseVector([0.0036, 0.0, 0.0, 0.0178, 0.0392, 0.0, 0.1852, 0.0178, 0.0, 0.0036, 0.0, 0.0249, 0.0142, 0.9792, 0.0605, 0.0071, 0.0071, 0.0, 0.0]))

In [46]:
#Model Training

lr = LogisticRegression(maxIter=5, regParam=0.0)
model = lr.fit(data)

In [47]:
model.coefficientMatrix

DenseMatrix(1, 19, [-1.4555, 64.3309, -50.2262, 53.163, 6.5853, 26.597, 7.4903, 4.0636, ..., 10.7532, -2.5073, -1.8697, -2.3707, -11.4573, 23.7186, -58.4153, 118.115], 1)

In [48]:
model.summary.accuracy

0.8044444444444444

In [49]:
#3 way split splits
train, rest = df.randomSplit([0.6, 0.4], seed=40)
test, validation = rest.randomSplit([0.5, 0.5], seed =40)
#train is now 60%, test is 20% and validation is 20%

In [50]:
#2 way split
train, test = df_features.drop('userId').randomSplit([0.8, 0.2], seed=40)

In [51]:
#pipeline

assembler = VectorAssembler(inputCols=numeric_feature_columns, outputCol="NumFeatures")
scaler = Normalizer(inputCol="NumFeatures", outputCol="features")
lr = LogisticRegression(maxIter=5, regParam=0.0)

pipeline = Pipeline(stages=[assembler, scaler, lr])

pipeline_model = pipeline.fit(train)



In [52]:
predictions = pipeline_model.transform(test)

In [53]:
predictions.head()

Row(is_female=0, event_Cancel=0, event_Submit_Downgrade=0, event_Thumbs_Down=6, event_Home=20, event_Downgrade=5, event_Roll_Advert=7, event_Logout=5, event_Save_Settings=0, label=0, event_About=3, event_Settings=3, event_Add_to_Playlist=8, event_Add_Friend=4, event_NextSong=387, event_Thumbs_Up=21, event_Help=2, event_Upgrade=2, event_Error=0, event_Submit_Upgrade=1, NumFeatures=DenseVector([0.0, 0.0, 0.0, 6.0, 20.0, 5.0, 7.0, 5.0, 0.0, 3.0, 3.0, 8.0, 4.0, 387.0, 21.0, 2.0, 2.0, 0.0, 1.0]), features=DenseVector([0.0, 0.0, 0.0, 0.0154, 0.0515, 0.0129, 0.018, 0.0129, 0.0, 0.0077, 0.0077, 0.0206, 0.0103, 0.9964, 0.0541, 0.0051, 0.0051, 0.0, 0.0026]), rawPrediction=DenseVector([0.8634, -0.8634]), probability=DenseVector([0.7034, 0.2966]), prediction=0.0)

In [54]:
predictions.filter(predictions.label == predictions.prediction).count()

39

In [ ]:
#crossval = CrossValidator(estimator = pipeline)

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.